In [75]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [76]:
# Tool Creation

@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers this tool returns their product"""
    return a*b

In [77]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [78]:
multiply.name

'multiply'

In [79]:
multiply.description

'Given 2 numbers this tool returns their product'

In [80]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [81]:
from dotenv import load_dotenv
load_dotenv()

True

In [135]:
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

In [136]:
llm_with_tools = model.bind_tools([multiply])

In [84]:
llm_with_tools.invoke('Hi how are you ?') 

AIMessage(content='Hi! I’m doing great, thanks for asking. How are you?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 134, 'total_tokens': 170}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--91931d72-423b-485f-b5f5-721fd2ada5a8-0', usage_metadata={'input_tokens': 134, 'output_tokens': 36, 'total_tokens': 170})

In [85]:
query = HumanMessage('can you multiply 3 with 10')

In [86]:
messages = [query]

In [87]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [88]:
result = llm_with_tools.invoke(messages)

In [89]:
messages.append(result)

In [90]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': 'call_gyyeb09DW5hDNevkaRbsQNOV', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 137, 'total_tokens': 182}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3f112781-05f8-44af-9608-88dfcc7102f7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_gyyeb09DW5hDNevkaRbsQNOV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 137, 'output_tokens': 45, 'total_tokens': 182})]

In [91]:
tool_result = multiply.invoke(result.tool_calls[0])

In [92]:
messages.append(tool_result)

In [93]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': 'call_gyyeb09DW5hDNevkaRbsQNOV', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 137, 'total_tokens': 182}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3f112781-05f8-44af-9608-88dfcc7102f7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_gyyeb09DW5hDNevkaRbsQNOV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 137, 'output_tokens': 45, 'total_tokens': 182}),
 ToolMessage(content='30', name='multiply', tool_call_id='call_gyyeb09DW5hDNevkaRbsQNOV')]

In [94]:
llm_with_tools.invoke(messages).content

'The result of multiplying\u202f3\u202fby 10 is **30**.'

In [213]:
# Tool Creation
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/40b6138d6d0b30711b7a524d/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Given a currency conversion rate this function calculates the target currency value from a given base currency value"""

    return base_currency_value*conversion_rate

In [214]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756512002,
 'time_last_update_utc': 'Sat, 30 Aug 2025 00:00:02 +0000',
 'time_next_update_unix': 1756598402,
 'time_next_update_utc': 'Sun, 31 Aug 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.0985}

In [215]:
convert.invoke({'base_currency_value':10, 'conversion_rate':88.0985})

880.985

In [216]:
llm = HuggingFaceEndpoint(
    repo_id="NousResearch/Hermes-4-405B",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

In [217]:
llm_with_tools = model.bind_tools([convert, get_conversion_factor])

In [218]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and also based on that please convert 10 USD to INR ?')]

In [219]:
ai_message = llm_with_tools.invoke(messages)

In [220]:
messages.append(ai_message)

In [221]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'chatcmpl-tool-2107271aa41d4d5189a11cb58f276280',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'chatcmpl-tool-d80810e629f3413a817c7fef5193498b',
  'type': 'tool_call'}]

In [222]:
import json
for tool_call in ai_message.tool_calls:
    # execute the 1st tool to get the value of conversion rate
    # Execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch the conversion rate 
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)

    if tool_call['name'] == 'convert':
        # fetch the current argument
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)



In [223]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and also based on that please convert 10 USD to INR ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'chatcmpl-tool-2107271aa41d4d5189a11cb58f276280', 'type': 'function'}, {'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert', 'description': None}, 'id': 'chatcmpl-tool-d80810e629f3413a817c7fef5193498b', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 289, 'total_tokens': 340}, 'model_name': 'NousResearch/Hermes-4-405B', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--254f378d-bd35-4173-98e2-d5651420e522-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'IN

In [224]:
result = llm_with_tools.invoke(messages)

In [225]:
result.content

'The conversion factor between USD and INR is 88.0985. Therefore, 10 USD is approximately 880.985 INR.'